# Word Analysis of 2020 Presidential Debates

## Import Packages

In [1]:

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords

from plotly.subplots import make_subplots
import plotly.graph_objects as go
#import plotly.express as px
#import plotly.figure_factory as ff

## Read Data

In [2]:
df1=pd.read_excel("Data_files/New_1st_debate.xlsx")
df=pd.read_excel("Data_files/New_2nd_debate.xlsx")
df.head()
df1.head()

speaker minute_parts  \
0  Chris Wallace     00:54:00   
1  Chris Wallace     02:10:00   
2      Joe Biden     02:49:00   
3   Donald Trump     02:51:00   
4      Joe Biden     02:51:00   

                                                text  minutes    minute  
0  Good evening from the Health Education Campus ...        1  00:00:54  
1  This debate is being conducted under health an...        1  00:01:34  
2                                How you doing, man?        2  00:02:13  
3                                 How are you doing?        2  00:02:15  
4                                          I’m well.        2  00:02:15

In [3]:
null_df = pd.DataFrame(pd.concat([df.isnull().sum()], axis = 1))
null_df.columns = ['df']
null_df

df
speaker   0
minute    0
text      0

In [4]:
df.shape

(513, 3)

## Heading check

In [5]:
print('names in the first dataset:', (df1.speaker.unique()))
print('names in the second dataset:', (df.speaker.unique()))

df1.loc[df1.speaker.str.contains('Chris Wallace:'), 'speaker'] = 'Chris Wallace' # correcting the typo in the name

# changing their names for more simplicity and coherence in two datasets
df1.loc[df1.speaker.str.contains('Vice President Joe Biden'), 'speaker'] = 'Joe Biden'
df1.loc[df1.speaker.str.contains('President Donald J. Trump'), 'speaker'] = 'Donald Trump'

df1.loc[df1.speaker.str.contains('Chris Wallace'), 'speaker'] = 'mediator_1'
df.loc[df.speaker.str.contains('Kristen Welker'), 'speaker'] = 'mediator_2'
print('Modified names in the first dataset:', (df1.speaker.unique()))
print('Modified names in the second dataset:', (df.speaker.unique()))

names in the first dataset: ['Chris Wallace' 'Joe Biden' 'Donald Trump']
names in the second dataset: ['Kristen Welker' 'Donald Trump' 'Joe Biden']
Modified names in the first dataset: ['mediator_1' 'Joe Biden' 'Donald Trump']
Modified names in the second dataset: ['mediator_2' 'Donald Trump' 'Joe Biden']


In [6]:
# we want to analyze the debate based on the sentences
# so we use a sentence level tokenizer from nltk
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

# number of sentences used by each person, each time their allowed to talk
df1['number_of_sents'] = df1.text.apply(lambda x:len(sent_detector.tokenize(x)))
df['number_of_sents'] = df.text.apply(lambda x:len(sent_detector.tokenize(x)))

In [7]:
number_of_sentences_1 = df1.groupby(['speaker']).sum()[['number_of_sents']].reset_index()
number_of_sentences_2 = df.groupby(['speaker']).sum()[['number_of_sents']].reset_index()
# total number of sentences in each debate
total_num_sents_1 = number_of_sentences_1.sum().number_of_sents 
total_num_sents_2 = number_of_sentences_2.sum().number_of_sents 
# percentage of conversation dominated by each candidate
# based on the number of sentences they used
number_of_sentences_1.loc[:, 'percentage'] = number_of_sentences_1.number_of_sents.apply(lambda x:round(x/total_num_sents_1, 2))
number_of_sentences_2.loc[:, 'percentage'] = number_of_sentences_2.number_of_sents.apply(lambda x:round(x/total_num_sents_2, 2))

number_of_sentences_1

speaker  number_of_sents  percentage
0  Donald Trump              796        0.42
1     Joe Biden              664        0.35
2    mediator_1              453        0.24

In [8]:
number_of_sentences_2

speaker  number_of_sents  percentage
0  Donald Trump              737        0.44
1     Joe Biden              554        0.33
2    mediator_2              383        0.23

In [9]:
# plot of the results
fig = make_subplots(rows=2, cols=2,
                    specs=[[{"rowspan": 2}, {}]
                            ,[None,         {}]],
                    subplot_titles=("# of sentences in total","First Debate", "Second Debate"))

fig.add_trace(go.Bar(x=['First Debate', 'Second Debate'], 
                     y=[total_num_sents_1, total_num_sents_2],
                     text =[total_num_sents_1, total_num_sents_2]),
                     row=1, col=1)

fig.add_trace(go.Bar(x=number_of_sentences_1.speaker,
                     y=number_of_sentences_1.number_of_sents,
                     text =number_of_sentences_1.percentage),
                     row=1, col=2)
fig.add_trace(go.Bar(x=number_of_sentences_2.speaker,
                     y=number_of_sentences_2.number_of_sents,
                     text =number_of_sentences_2.percentage),
                     row=2, col=2)

fig.update_traces(textposition='outside', textfont_size=14)
fig.update_layout(showlegend=False, title_text="Number of sentences in Both debates")
fig.update_yaxes(title_text='count')
fig.show()